In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import random
from operator import attrgetter
from scipy.stats import poisson

In [2]:
Group_A = ["Paris Saint-Germain>FRA","Real Madrid CF>ESP","Club Brugge>BEL","Galatasaray İstanbul AŞ>TUR"]
Group_B = ["Bayern München>GER","Tottenham Hotspur FC>ENG","Olympiacos>GRE","Crvena Zvezda>SER"]
Group_C = ["Manchester City FC>ENG","Shakhtar Donetsk>UKR","Dinamo Zagreb>CRO","Atalanta BC>ITA"]
Group_D = ["Juventus>ITA","Atletico Madrid>ESP","Bayer 04 Leverkusen>GER","Lokomotiv Moskva>RUS"]
Group_E = ["Liverpool FC>ENG","SSC Napoli>ITA","FC RB Salzburg>AUT","KRC Genk>BEL"]
Group_F = ["FC Barcelona>ESP","Borussia Dortmund>GER","Internazionale>ITA","Slavia Praha>CZK"]
Group_G = ["Zenit St. Petersburg>RUS","SL Benfica>POR","Olympique Lyon>FRA","RB Leipzig>GER"]
Group_H = ["Chelsea FC>ENG","AFC Ajax>NED","Valencia CF>ESP","Lille OSC>FRA"]
Groups =  [Group_A,Group_B,Group_C,Group_D,Group_E,Group_F,Group_G,Group_H]
T = []
for group in Groups:
    T.extend(group)

In [3]:
coeffs = pd.read_excel("coeff.xlsx")
coeffs["Club"] = coeffs["Club"].str.replace("FC Santa Coloma",'E Santa Coloma')
coeffs["Club"] = coeffs["Club"].str.replace("FC RB Salzburg",'E RB Salzburg')
coeffs["Club"] = coeffs["Club"].str.replace("AFC Ajax","E Ajax")
coeffs["Club"] = coeffs["Club"].str.replace("Celtic FC",'E Celtic')
coeffs["Club"] = coeffs["Club"].str.replace("Panathinaikos FC",'E Panathinaikos')
coeffs["Club"] = coeffs["Club"].str.replace("SS Folgore",'S.S. Folgore Falciano Calcio')
coeffs["Club"] = coeffs["Club"].str.replace(" FC",'')
coeffs["Club"] = coeffs["Club"].str.replace("FC ",'')
coeffs["Club"] = coeffs["Club"].str.replace("E Santa Coloma",'FC Santa Coloma')
coeffs["Club"] = coeffs["Club"].str.replace("E Celtic",'Celtic FC')
coeffs["Club"] = coeffs["Club"].str.replace("E Panathinaikos",'Panathinaikos FC')
coeffs["Club"] = coeffs["Club"].str.replace("E RB Salzburg",'FC RB Salzburg')
coeffs["Club"] = coeffs["Club"].str.replace("E Ajax","AFC Ajax")
coeffs["Club"] = coeffs["Club"].str.replace("Manchester United","Manchester United FC")
coeffs["Club"] = coeffs["Club"].str.replace("Manchester City","Manchester City FC")
coeffs["Club"] = coeffs["Club"].str.replace("Sevilla","Sevilla FC")
coeffs["Club"] = coeffs["Club"].str.replace("Chelsea","Chelsea FC")
coeffs["Club"] = coeffs["Club"].str.replace("Barcelona","FC Barcelona")
coeffs["Club"] = coeffs["Club"].str.replace("Arsenal","Arsenal FC")
coeffs["Club"] = coeffs["Club"].str.replace("Porto","FC Porto")
coeffs["Club"] = coeffs["Club"].str.replace("Liverpool","Liverpool FC")
coeffs["Club"] = coeffs["Club"].str.replace("Schalke 04","FC Schalke 04")
coeffs["Club"] = coeffs["Club"].str.replace("Neftchi PBaku","Neftchi PFC Baku")
coeffs["Club"] = coeffs["Club"].str.replace("PAOK","PAOK FC")
coeffs["Club"] = coeffs["Club"].str.replace("Motherwell","Motherwell FC")
coeffs["Club"] = coeffs["Club"].str.replace("PLitex Lovech","PFC Litex Lovech")
coeffs["Club"] = coeffs["Club"].str.replace("Rangers","Rangers FC")
coeffs["Club"] = coeffs["Club"].str.replace("Twente","FC Twente")
coeffs["Club"] = coeffs["Club"].str.replace("Zurich","FC Zurich")
coeffs["Club"] = coeffs["Club"].str.replace("Tottenham Hotspur","Tottenham Hotspur FC")
coeffs["Club"] = coeffs["Club"].str.replace("Politehnica Timisoara","FC Politehnica Timisoara")
coeffs["Club"] = coeffs["Club"].str.replace("Toulouse","Toulouse FC")
coeffs["Club"] = coeffs["Club"].str.replace("Everton","Everton FC")
coeffs["Club"] = coeffs["Club"].str.replace("PLokomotiv Plovdiv","PFC Lokomotiv Plovdiv")
coeffs["Club"] = coeffs["Club"].str.replace("Grasshoppers FC Zurich","Grasshoppers Zurich")
coeffs["Club"] = coeffs["Club"].str.replace("Newcastle United","Newcastle United FC")
coeffs["Club"] = coeffs["Club"].str.replace("Boavista","Boavista FC")


In [4]:
coeff = coeffs.set_index("Club")

In [5]:
class team:
    def __init__(self, name, points=0, wins=0, draws=0, losses=0,goaldiff=0,nation = ""):

        self.name = name
        self.points = points
        self.wins = wins
        self.draws = draws
        self.losses = losses
        self.goaldiff = goaldiff
        self.coeff = coeff.loc[self.name].loc['Pts']
        self.nation = nation
        
    def __repr__(self):
        return repr((self.name))

In [6]:
Team = {}
for group in Groups:
    for t in group:
        e = t.split('>')
        #print(e)
        team_name = e[0]
        country = e[1]
        Team[team_name] = team(name = team_name ,nation = country)



In [7]:
def old_simulate_group_stage_draw(Team):
    sorted_teams = sorted(Team.values(),key = attrgetter('coeff'),reverse=True)
    pot1 = sorted_teams[:8]
    pot2 = sorted_teams[8:16]
    pot3 = sorted_teams[16:24]
    pot4 = sorted_teams[24:]
    Group={}
    Country = {}
    flag = True
    avail_grps= [1,2,3,4,5,6,7,8]
    for t in pot1:
        i = random.choice(avail_grps)
        Group[i] = [t.name]
        Country[i] = [t.nation]
        avail_grps.remove(i)
    avail_grps= [1,2,3,4,5,6,7,8]
    for t in pot2:
        i = random.choice(avail_grps)
        if t.nation not in Country[i]:
            Group[i].append(t.name)
            Country[i].append(t.nation)
            avail_grps.remove(i)
        else:
            cnt = 0
            while(t.nation in Country[i]):
                i = random.choice(avail_grps)
                cnt+=1
                if cnt>8:
                    return Group,Country,False
                if t.nation not in Country[i]:
                    Group[i].append(t.name)
                    Country[i].append(t.nation)
                    avail_grps.remove(i)
                    break
    avail_grps= [1,2,3,4,5,6,7,8]
    for t in pot3:
        i = random.choice(avail_grps)
        if t.nation not in Country[i]:
            Group[i].append(t.name)
            Country[i].append(t.nation)
            avail_grps.remove(i)
        else:
            cnt = 0
            while(t.nation in Country[i]):
                i = random.choice(avail_grps)
                cnt+=1
                if cnt>8:
                    return Group,Country,False
                if t.nation not in Country[i]:
                    Group[i].append(t.name)
                    Country[i].append(t.nation)
                    avail_grps.remove(i)
                    break
    avail_grps= [1,2,3,4,5,6,7,8]
    
    for t in pot4:
        i = random.choice(avail_grps)
        if t.nation not in Country[i]:
            Group[i].append(t.name)
            Country[i].append(t.nation)
            avail_grps.remove(i)
        else:
            cnt = 0
            while(t.nation in Country[i]):
                i = random.choice(avail_grps)
                cnt+=1
                if cnt>8:
                    return Group,Country,False
                if t.nation not in Country[i]:
                    Group[i].append(t.name)
                    Country[i].append(t.nation)
                    avail_grps.remove(i)
                    break
    return Group,Country,flag
    


In [8]:
def new_simulate_group_stage_draw(Team):
    pot1 = [Team[group[0].split('>')[0]] for group in Groups]
    pot2 = [Team[group[1].split('>')[0]] for group in Groups] 
    pot3 = [Team[group[2].split('>')[0]] for group in Groups]
    pot4 = [Team[group[3].split('>')[0]] for group in Groups]
    Group={}

    Country = {}
    flag = True
    avail_grps= [1,2,3,4,5,6,7,8]
    for t in pot1:
        i = random.choice(avail_grps)
        Group[i] = [t.name]
        Country[i] = [t.nation]
        avail_grps.remove(i)
    avail_grps= [1,2,3,4,5,6,7,8]
    for t in pot2:
        i = random.choice(avail_grps)
        if t.nation not in Country[i]:
            Group[i].append(t.name)
            Country[i].append(t.nation)
            avail_grps.remove(i)
        else:
            cnt = 0
            while(t.nation in Country[i]):
                i = random.choice(avail_grps)
                cnt+=1
                if cnt>8:
                    return Group,Country,False
                if t.nation not in Country[i]:
                    Group[i].append(t.name)
                    Country[i].append(t.nation)
                    avail_grps.remove(i)
                    break
    avail_grps= [1,2,3,4,5,6,7,8]
    for t in pot3:
        i = random.choice(avail_grps)
        if t.nation not in Country[i]:
            Group[i].append(t.name)
            Country[i].append(t.nation)
            avail_grps.remove(i)
        else:
            cnt = 0
            while(t.nation in Country[i]):
                i = random.choice(avail_grps)
                cnt+=1
                if cnt>8:
                    return Group,Country,False
                if t.nation not in Country[i]:
                    Group[i].append(t.name)
                    Country[i].append(t.nation)
                    avail_grps.remove(i)
                    break
    avail_grps= [1,2,3,4,5,6,7,8]
    
    for t in pot4:
        i = random.choice(avail_grps)
        if t.nation not in Country[i]:
            Group[i].append(t.name)
            Country[i].append(t.nation)
            avail_grps.remove(i)
        else:
            cnt = 0
            while(t.nation in Country[i]):
                i = random.choice(avail_grps)
                cnt+=1
                if cnt>8:
                    return Group,Country,False
                if t.nation not in Country[i]:
                    Group[i].append(t.name)
                    Country[i].append(t.nation)
                    avail_grps.remove(i)
                    break
    return Group,Country,flag


In [9]:
def random_simulate_group_stage_draw(Team):
    clubs = [t.split('>')[0] for t in T]
    random.shuffle(clubs) 
    Country={}
    Group = {}
    flag = True
    for i in range(1,9):
        Group[i] = clubs[4*(i-1):4*i]
    return Group,Country,flag

In [10]:
def simulate_group(Group,home_coeff,away_coeff,home_c,away_c,final):
    for home in Group:
        for away in Group:
            if home == away:
                continue
            else:    
                home_team = Team[home]
                away_team = Team[away]
                home_theta = home_coeff*home_team.coeff + away_coeff*away_team.coeff+home_c*1+away_c*0+final*0
                away_theta = home_coeff*away_team.coeff + away_coeff*home_team.coeff+home_c*0+away_c*1+final*0
                home_theta = max(0,home_theta)
                away_theta = max(0,away_theta)
                home_score = poisson.rvs(home_theta,1)
                away_score = poisson.rvs(away_theta,1)
                if home_score > away_score:
                    home_team.wins+=1
                    home_team.points+=3
                    away_team.losses+=1
                elif home_score < away_score:
                    away_team.wins+=1
                    away_team.points+=3
                    home_team.losses+=1
                else:
                    away_team.draws+=1
                    away_team.points+=1
                    home_team.draws+=1
                    home_team.points+=1
                home_team.goaldiff += (home_score-away_score)
                away_team.goaldiff += (-1*(home_score-away_score))

In [11]:
def simulate_group_stages(Groups,home_coeff,away_coeff,home_c,away_c,final):
    Winner = []
    Second =[]

    for group in Groups:
        simulate_group(group,home_coeff,away_coeff,home_c,away_c,final)
        Group_Team = [Team[t] for t in group]
        Group_Team = sorted(Group_Team,key = attrgetter('points','goaldiff','wins'),reverse=True)
        Winner.append(Group_Team[0])
        Second.append(Group_Team[1])
    return Winner,Second

In [12]:
def simulate_round_of_16(Winner,Second,home_coeff,away_coeff,home_c,away_c,final):
    Qualified = []
    for i in range(8):
        w_i = random.choice(Winner)
        s_i = random.choice(Second)
        Winner.remove(w_i)
        Second.remove(s_i)
        home_team = Team[w_i.name]
        away_team = Team[s_i.name]
        leg1_home_theta = home_coeff*home_team.coeff + away_coeff*away_team.coeff+home_c*1+away_c*0+final*0
        leg1_away_theta = home_coeff*away_team.coeff + away_coeff*home_team.coeff+home_c*0+away_c*1+final*0
        leg2_home_theta = home_coeff*home_team.coeff + away_coeff*away_team.coeff+home_c*0+away_c*1+final*0
        leg2_away_theta = home_coeff*away_team.coeff + away_coeff*home_team.coeff+home_c*1+away_c*0+final*0
        leg1_home_theta = max(0,leg1_home_theta)
        leg1_away_theta = max(0,leg1_away_theta)
        leg2_home_theta = max(0,leg1_home_theta)
        leg2_away_theta = max(0,leg1_away_theta)
        home_score_leg_1 = poisson.rvs(leg1_home_theta,1)
        away_score_leg_1 = poisson.rvs(leg1_away_theta,1)
        home_score_leg_2 = poisson.rvs(leg2_home_theta,1)
        away_score_leg_2 = poisson.rvs(leg2_away_theta,1)
        if home_score_leg_2+home_score_leg_1>away_score_leg_1+away_score_leg_2:
            Qualified.append(w_i)
        elif home_score_leg_2+home_score_leg_1 < away_score_leg_1+away_score_leg_2:
            Qualified.append(s_i)
        elif leg1_home_theta+leg2_home_theta>leg1_away_theta+leg2_away_theta:
            Qualified.append(w_i)
        else:
            Qualified.append(s_i)

                
    return Qualified

            
        

In [13]:
def Quaters(Teams,home_coeff,away_coeff,home_c,away_c,final):
    Qualified = []
    for i in range(4):
        w_i = random.choice(Teams)
        Teams.remove(w_i)
        s_i = random.choice(Teams)
        Teams.remove(s_i)
        home_team = Team[w_i.name]
        away_team = Team[s_i.name]
        leg1_home_theta = home_coeff*home_team.coeff + away_coeff*away_team.coeff+home_c*1+away_c*0+final*0
        leg1_away_theta = home_coeff*away_team.coeff + away_coeff*home_team.coeff+home_c*0+away_c*1+final*0
        leg2_home_theta = home_coeff*home_team.coeff + away_coeff*away_team.coeff+home_c*0+away_c*1+final*0
        leg2_away_theta = home_coeff*away_team.coeff + away_coeff*home_team.coeff+home_c*1+away_c*0+final*0
        leg1_home_theta = max(0,leg1_home_theta)
        leg1_away_theta = max(0,leg1_away_theta)
        leg2_home_theta = max(0,leg1_home_theta)
        leg2_away_theta = max(0,leg1_away_theta)
        home_score_leg_1 = poisson.rvs(leg1_home_theta,1)
        away_score_leg_1 = poisson.rvs(leg1_away_theta,1)
        home_score_leg_2 = poisson.rvs(leg2_home_theta,1)
        away_score_leg_2 = poisson.rvs(leg2_away_theta,1)
        if home_score_leg_2+home_score_leg_1>away_score_leg_1+away_score_leg_2:
            Qualified.append(w_i)
        elif home_score_leg_2+home_score_leg_1 < away_score_leg_1+away_score_leg_2:
            Qualified.append(s_i)
        elif leg1_home_theta+leg2_home_theta>leg1_away_theta+leg2_away_theta:
            Qualified.append(w_i)
        else:
            Qualified.append(s_i)
    
    return Qualified

        



In [14]:
def Semi(Teams,home_coeff,away_coeff,home_c,away_c,final):
    Qualified = []
    for i in range(2):
        w_i = random.choice(Teams)
        Teams.remove(w_i)
        s_i = random.choice(Teams)
        Teams.remove(s_i)
        home_team = Team[w_i.name]
        away_team = Team[s_i.name]
        leg1_home_theta = home_coeff*home_team.coeff + away_coeff*away_team.coeff+home_c*1+away_c*0+final*0
        leg1_away_theta = home_coeff*away_team.coeff + away_coeff*home_team.coeff+home_c*0+away_c*1+final*0
        leg2_home_theta = home_coeff*home_team.coeff + away_coeff*away_team.coeff+home_c*0+away_c*1+final*0
        leg2_away_theta = home_coeff*away_team.coeff + away_coeff*home_team.coeff+home_c*1+away_c*0+final*0
        leg1_home_theta = max(0,leg1_home_theta)
        leg1_away_theta = max(0,leg1_away_theta)
        leg2_home_theta = max(0,leg1_home_theta)
        leg2_away_theta = max(0,leg1_away_theta)
        
        home_score_leg_1 = poisson.rvs(leg1_home_theta,1)
        away_score_leg_1 = poisson.rvs(leg1_away_theta,1)
        home_score_leg_2 = poisson.rvs(leg2_home_theta,1)
        away_score_leg_2 = poisson.rvs(leg2_away_theta,1)
        if home_score_leg_2+home_score_leg_1>away_score_leg_1+away_score_leg_2:
            Qualified.append(w_i)
        elif home_score_leg_2+home_score_leg_1 < away_score_leg_1+away_score_leg_2:
            Qualified.append(s_i)
        elif leg1_home_theta+leg2_home_theta>leg1_away_theta+leg2_away_theta:
            Qualified.append(w_i)
        else:
            Qualified.append(s_i)

    return Qualified


In [15]:
def Final(Teams,home_coeff,away_coeff,home_c,away_c,final):
    home_team = Team[Teams[0].name]
    away_team = Team[Teams[1].name]
    leg1_home_theta = home_coeff*home_team.coeff + away_coeff*away_team.coeff+home_c*0+away_c*0+final*1
    leg1_away_theta = home_coeff*away_team.coeff + away_coeff*home_team.coeff+home_c*0+away_c*0+final*1
    leg1_home_theta = max(0,leg1_home_theta)
    leg1_away_theta = max(0,leg1_away_theta)
    home_score_leg_1 = poisson.rvs(leg1_home_theta,1)
    away_score_leg_1 = poisson.rvs(leg1_away_theta,1)
    #leg2_home_theta = max(0,leg1_home_theta)
    #leg2_away_theta = max(0,leg1_away_theta)
        
    if home_score_leg_1>away_score_leg_1:
        return [Teams[0]]
    elif home_score_leg_1 < away_score_leg_1:
         return [Teams[1]]
    elif leg1_home_theta>leg1_away_theta:
         return [Teams[0]]
    else:
        return [Teams[1]]

In [16]:
#Group_qualified={}
#Rof_16_qualified = {}
#Quaters_qualified = {}
#Semi_Finals_Qaulified = {}
#Winners = {}

In [17]:

#Group,Country,F = old_simulate_group_stage_draw(Team)
#Group_qualified={}
#while F == False:
#    Group,Country,F = old_simulate_group_stage_draw(Team)
#Winner,Second = simulate_group_stages(Group.values())
#qual = []
#qual.extend(Second)
#qual.extend(Winner)
#for t in qual:
#    val = Group_qualified.get(t,0)
#    val+=1
#    Group_qualified[t] = val
#Qualified = simulate_round_of_16(Winner,Second)
#semi  = Quaters(Qualified)
#finalists = Semi(semi)
#winner = Final(finalists)

In [18]:
def one_mcmc(home_coeff,away_coeff,home,away,final,Group_qualified = {},Rof_16_qualified={},Quaters_qualified={},Semi_Finals_Qaulified={},Winners={},i_ter= 1000):
    for i in range(i_ter):
        Group,Country,F = old_simulate_group_stage_draw(Team)
    #Group_qualified={}
        while F == False:
            Group,Country,F = old_simulate_group_stage_draw(Team)
        Winner,Second = simulate_group_stages(Group.values(),home_coeff,away_coeff,home,away,final)
        qual = []
        qual.extend(Second)
        qual.extend(Winner)
        for t in qual:
            val = Group_qualified.get(t,0)
            val+=1
            Group_qualified[t] = val
        Qualified = simulate_round_of_16(Winner,Second,home_coeff,away_coeff,home,away,final)
        for t in Qualified:
            val = Rof_16_qualified.get(t,0)
            val+=1
            Rof_16_qualified[t] = val
    
        semi  = Quaters(Qualified,home_coeff,away_coeff,home,away,final)
        for t in semi:
            val = Quaters_qualified.get(t,0)
            val+=1
            Quaters_qualified[t] = val
        finalists = Semi(semi,home_coeff,away_coeff,home,away,final)
        for t in finalists:
            val = Semi_Finals_Qaulified.get(t,0)
            val+=1
            Semi_Finals_Qaulified[t] = val
        winner = Final(finalists,home_coeff,away_coeff,home,away,final)
        for t in winner:
            val = Winners.get(t,0)
            val+=1
            Winners[t] = val
    return Group_qualified,Rof_16_qualified,Quaters_qualified,Semi_Finals_Qaulified,Winners



In [19]:
params = pd.read_csv("coefficients.csv",index_col=0)

In [28]:
Gp_graph = {}
cnt =0
for index, row in params[-20:].iterrows():
    home_coeff,away_coeff,home,away,final = float(row['Home_coeff_samples']), float(row['Away_coeff_samples']),float(row['Home_samples']),float(row['Away_samples']),float(row['Final_samples'])
    print(home_coeff,away_coeff,home,away,final)
    Group_qualified,Rof_16_qualified,Quaters_qualified,Semi_Finals_Qaulified,Winners = one_mcmc(home_coeff,away_coeff,home,away,final,{},{},{},{},{})
    for k,v in Group_qualified.items():
        ex = Gp_graph.get(k,[])
        ex.append(v/1000)
        Gp_graph[k] = ex 
    print(cnt)
    cnt+=1
    if cnt>20:
        break
    

0.0005624424677488 -0.0004099860486887 0.4101957263326878 0.0636310803197538 0.1411940677509393
0
0.0007604419454861 -0.0004805147676035 0.4108469083180423 0.0678385665628807 0.5035616552729383
1
0.000625184914226 -0.0004612656895009 0.4103776086927272 0.0660323627354219 0.2487580054221971
2
0.0006104168807797 -0.0004552662542769 0.4115760599280774 0.0614952260092336 0.2623993795616817
3
0.0006199662327175 -0.0003404962896506 0.4121282356717398 0.064436156798821 0.2586237630611043
4
0.000700673522994 -0.0005011690743203 0.4072302913541848 0.0872738158124416 0.1383525114155857
5
0.0006753653945315 -0.0003499534913322 0.4059979284896513 0.0353422532748707 0.4830645653158798
6
0.0006005806337853 -0.0003560572372769 0.4059327425633775 0.0638149242621787 0.3835933558596489
7
0.000527203765829 -0.0002855027707443 0.4060384058223044 0.0568235504264531 0.1622037957843152
8
0.000720408320499 -0.0005878282882279 0.4060888854499873 0.0753818387918578 0.4017571452991795
9
0.0003855353366826 -0.000

In [30]:
Gp_graph

{'Tottenham Hotspur FC': [1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0],
 'Olympique Lyon': [1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0],
 'Shakhtar Donetsk': [1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0],
 'Atletico Madrid': [1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0],
 'RB Leipzig': [1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0],
 'AFC Ajax': [1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0,
  1.0],
 'Borussia Dortmund': [1.0,
  1.0,
  1.0,
  1.0